In [5]:
import requests
import pandas as pd

fmarket_headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "vi",
    "content-type": "application/json",
    "sec-ch-ua": "\"Microsoft Edge\";v=\"119\", \"Chromium\";v=\"119\", \"Not?A_Brand\";v=\"24\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "referrer": "https://fmarket.vn/"
}

payload = {
    "types": ["NEW_FUND", "TRADING_FUND"],
    "issuerIds": [],
    "sortOrder": "DESC",
    "sortField": "navTo6Months",
    "page": 1,
    "pageSize": 100,
    "isIpo": False,
    "fundAssetTypes": [],
    "bondRemainPeriods": [],
    "searchField": "",
    "isBuyByReward": False,
    "thirdAppIds": []
}

url = "https://api.fmarket.vn/res/products/filter"
response = requests.post(url, json=payload, headers=fmarket_headers)
status = response.status_code

if status != 200:
    raise requests.exceptions.HTTPError(
        f"Error in API response: {status} - {response.text}"
    )

data = response.json()
print('Total number of funds: ', data['data']['total'])

df = pd.json_normalize(data, record_path=['data', 'rows'])

# select columns to display
column_subset = [
    'id',
    'shortName',
    'name',
    'owner.name',
    'dataFundAssetType.name',
    'firstIssueAt',
    'managementFee',
    'nav',
    'productNavChange.navToPrevious',
    'productNavChange.navToLastYear',
    'productNavChange.navToBeginning',
    'productNavChange.navTo1Months',
    'productNavChange.navTo3Months',
    'productNavChange.navTo6Months',
    'productNavChange.navTo12Months',
    'productNavChange.navTo24Months',
    'productNavChange.navTo36Months',
    'productNavChange.annualizedReturn36Months',
    'productNavChange.updateAt'            
]
df = df[column_subset]

# sort by column
df = df.sort_values('shortName')

# remove index column
df = df.reset_index(drop=True)

# output
print(df.head(5))
#df.to_csv('fmarket.csv', index=False, encoding='utf-8')

# compare pyarrow vs numpy backend dataframe
df.info()
df = df.convert_dtypes(dtype_backend="pyarrow")
df.info()



Total number of funds:  44
   id shortName                                     name  \
0  65      ABBF            QUỸ ĐẦU TƯ TRÁI PHIẾU AN BÌNH   
1  51      ASBF      QUỸ ĐẦU TƯ TRÁI PHIẾU AN TOÀN AMBER   
2  13      BVBF          QUỸ ĐẦU TƯ TRÁI PHIẾU BẢO VIỆT    
3  12     BVFED   QUỸ ĐẦU TƯ CỔ PHIẾU NĂNG ĐỘNG BẢO VIỆT   
4  14      BVPF  QUỸ ĐẦU TƯ CỔ PHIẾU TRIỂN VỌNG BẢO VIỆT   

                                          owner.name dataFundAssetType.name  \
0  CÔNG TY CỔ PHẦN QUẢN LÝ QUỸ ĐẦU TƯ CHỨNG KHOÁN...         Quỹ trái phiếu   
1                  CÔNG TY CỔ PHẦN QUẢN LÝ QUỸ AMBER         Quỹ trái phiếu   
2                  CÔNG TY TNHH QUẢN LÝ QUỸ BẢO VIỆT         Quỹ trái phiếu   
3                  CÔNG TY TNHH QUẢN LÝ QUỸ BẢO VIỆT           Quỹ cổ phiếu   
4                  CÔNG TY TNHH QUẢN LÝ QUỸ BẢO VIỆT           Quỹ cổ phiếu   

    firstIssueAt  managementFee       nav  productNavChange.navToPrevious  \
0  1604336400000            0.0  12427.75                   